## IMPORTAR LAS LIBRERIAS

In [1]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
ruta_proyecto = 'E:/alex/disco duro viejo/Desarrollo profesional/Python/DS4B/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/01_LEADSCORING'

### Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'Leads.csv'

### Cargar los datos

In [4]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa, index_col = 'id', sep = ';')

### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [5]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['ambito_Select',
 'descarga_lm_No',
 'fuente_Organic Search',
 'ocupacion_Unemployed',
 'ocupacion_Working Professional',
 'paginas_vistas_visita_mms',
 'score_actividad_mms',
 'score_perfil_mms',
 'tiempo_en_site_total_mms',
 'ult_actividad_Chat Conversation',
 'ult_actividad_Converted to Lead',
 'ult_actividad_Page Visited on Website',
 'ult_actividad_SMS Sent',
 'visitas_total_mms']

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [6]:
variables_finales = ['ambito',
                     'ocupacion',
                     'descarga_lm',
                     'paginas_vistas_visita',
                     'score_actividad',
                     'score_perfil',
                     'tiempo_en_site_total',
                     'ult_actividad',
                     'visitas_total']

## ESTRUCTURA DE LOS DATASETS

### Eliminar registros

#### Por duplicados

In [7]:
df.drop_duplicates(inplace = True)

#### Por EDA

In [8]:
df = df.loc[(df.no_enviar_email != 'Yes') & (df.no_llamar != 'OTROS') & (df.ult_actividad != 'Email Bounced')]

#### Para x

Quedarse solo con las de la lista.

In [9]:
x = df[variables_finales].copy()

In [10]:
x

,ambito,ocupacion,descarga_lm,paginas_vistas_visita,score_actividad,score_perfil,tiempo_en_site_total,ult_actividad,visitas_total
id,,,,,,,,,
660737,Select,Unemployed,No,0.0,15.0,15.0,0,Page Visited on Website,0.0
660728,Select,Unemployed,No,2.5,15.0,15.0,674,Email Opened,5.0
660727,Business Administration,Student,Yes,2.0,14.0,20.0,1532,Email Opened,2.0
660719,Media and Advertising,Unemployed,No,1.0,13.0,17.0,305,Unreachable,1.0
660681,Select,Unemployed,No,1.0,15.0,18.0,1428,Converted to Lead,2.0
...,...,...,...,...,...,...,...,...,...
579622,Select,Unemployed,No,2.6,NaN,NaN,1409,SMS Sent,13.0
579615,Business Administration,Unemployed,No,2.5,14.0,20.0,210,SMS Sent,5.0
579546,Media and Advertising,Unemployed,Yes,2.0,14.0,19.0,238,SMS Sent,2.0


#### Para y

Especificar la target.

In [11]:
target = 'compra'

Crear el y.

In [12]:
y = df[target].copy()

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [13]:
def calidad_datos(df):
    temp = df.astype({'visitas_total': 'Int64'}) 
    
    #Creamos la función para imputar por la moda en 2 variables categóricas
    def imputar_moda(variable):
        return(variable.fillna(variable.mode()[0]))
    
    var_imputar_moda = ['ocupacion','ambito']
    
    #imputamos por moda
    temp[var_imputar_moda] = temp[var_imputar_moda].apply(imputar_moda)
    
    
    #imputamos por valor el resto de var categóricas que preseleccionaremos después
    var_imputar_valor = ['descarga_lm','ult_actividad']
    
    valor = 'DESCONOCIDO'
    
    #Imputamos por valor
    temp[var_imputar_valor] = temp[var_imputar_valor].fillna(valor)
    
    
    #imputamos por mediana
    var_imputar_mediana = ['paginas_vistas_visita',
                           'score_actividad',
                           'score_perfil',
                           'tiempo_en_site_total',
                           'visitas_total']
    #Definimos la función
    def imputar_mediana(variable):
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))
    
    #Realizamos la imputación por la mediana
    temp[var_imputar_mediana] = temp[var_imputar_mediana].apply(imputar_mediana)
    
    
    #Atípicos para categóricas
    def agrupar_cat_raras(variable, criterio = 0.019):
        #Calcula las frecuencias
        frecuencias = variable.value_counts(normalize=True)
        #Identifica las que están por debajo del criterio
        temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
        #Las recodifica en 'OTROS'
        temp2 = np.where(variable.isin(temp),'OTROS',variable)
        #Devuelve el resultado
        return(temp2)
    
    #Definimos las variables cat
    var_agrupar_cat_raras = ['ocupacion','ambito','descarga_lm','ult_actividad']
    
    #Aplicamos
    for variable in var_agrupar_cat_raras:
        temp[variable] = agrupar_cat_raras(temp[variable],criterio = 0.02)
        
    #Atípicos para numéricas por winsorización manual
    temp['visitas_total'].clip(0,50, inplace=True)
    temp['paginas_vistas_visita'].clip(0, 20, inplace=True)
    
    return(temp)

#### Convertirla en transformer

In [14]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

### Instanciar transformación de variables

In [15]:
#Variables a aplicar OHE
var_ohe = ['ambito','descarga_lm','ocupacion','ult_actividad']
#Instanciamos
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')



var_mms = ['paginas_vistas_visita','score_actividad','score_perfil','tiempo_en_site_total','visitas_total']
#instanciamos
mms = MinMaxScaler()
#Entrenamos y aplicamos
#df_mms = mms.fit_transform(df[var_mms])

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [16]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (mms, var_mms),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [17]:
pipe_prepro = make_pipeline(hacer_calidad_datos, 
                            ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [18]:
modelo = LogisticRegression(n_jobs = -1, 
                           solver = 'saga',
                           penalty = 'none',
                           C = 0)

#### Crear el pipe final de entrenamiento

In [19]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Guardar el pipe final de entrenamiento

In [20]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [21]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

C:\Users\Alex\miniconda3\envs\proyecto1\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [22]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [23]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)